In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from binance_historical_data import BinanceDataDumper

data_dumper = BinanceDataDumper(
    path_dir_where_to_dump=".",
    asset_class="spot",  # spot, um, cm
    data_type="klines",  # aggTrades, klines, trades
    data_frequency="1m",
)

print(data_dumper.get_list_all_trading_pairs())

data_dumper.dump_data(tickers=["NEARUSDT"])

['ETHBTC', 'LTCBTC', 'BNBBTC', 'NEOBTC', 'QTUMETH', 'EOSETH', 'SNTETH', 'BNTETH', 'BCCBTC', 'GASBTC', 'BNBETH', 'BTCUSDT', 'ETHUSDT', 'HSRBTC', 'OAXETH', 'DNTETH', 'MCOETH', 'ICNETH', 'MCOBTC', 'WTCBTC', 'WTCETH', 'LRCBTC', 'LRCETH', 'QTUMBTC', 'YOYOBTC', 'OMGBTC', 'OMGETH', 'ZRXBTC', 'ZRXETH', 'STRATBTC', 'STRATETH', 'SNGLSBTC', 'SNGLSETH', 'BQXBTC', 'BQXETH', 'KNCBTC', 'KNCETH', 'FUNBTC', 'FUNETH', 'SNMBTC', 'SNMETH', 'NEOETH', 'IOTABTC', 'IOTAETH', 'LINKBTC', 'LINKETH', 'XVGBTC', 'XVGETH', 'SALTBTC', 'SALTETH', 'MDABTC', 'MDAETH', 'MTLBTC', 'MTLETH', 'SUBBTC', 'SUBETH', 'EOSBTC', 'SNTBTC', 'ETCETH', 'ETCBTC', 'MTHBTC', 'MTHETH', 'ENGBTC', 'ENGETH', 'DNTBTC', 'ZECBTC', 'ZECETH', 'BNTBTC', 'ASTBTC', 'ASTETH', 'DASHBTC', 'DASHETH', 'OAXBTC', 'ICNBTC', 'BTGBTC', 'BTGETH', 'EVXBTC', 'EVXETH', 'REQBTC', 'REQETH', 'VIBBTC', 'VIBETH', 'HSRETH', 'TRXBTC', 'TRXETH', 'POWRBTC', 'POWRETH', 'ARKBTC', 'ARKETH', 'YOYOETH', 'XRPBTC', 'XRPETH', 'MODBTC', 'MODETH', 'ENJBTC', 'ENJETH', 'STORJBTC', 'ST

Tickers:   0%|          | 0/1 [00:00<?, ?it/s]

monthly files to download: 0files [00:00, ?files/s]

daily files to download: 0files [00:00, ?files/s]

Tried to dump data for 1 tickers:
---> For NEARUSDT new data saved for: 0 months 0 days


In [3]:
import pandas as pd
import numpy as np
import os

filenames = next(os.walk("./spot/monthly/klines/NEARUSDT/1m"), (None, None, []))[2]  # [] if no file

columns = [
    "Open time",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore"
]

df = pd.DataFrame(columns=columns)

for f in filenames:
    new_df = pd.read_csv(f"./spot/monthly/klines/NEARUSDT/1m/{f}", header=None, names=columns)
    df = pd.concat([df, new_df], ignore_index=True)
df = df.sort_values(by="Open time")
df

/tmp/ipykernel_46510/2473667814.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
388571,1602651600000,1.3000,1.8800,1.3000,1.3760,1429560.44,1602651659999,2.313165e+06,1848,520043.50,837306.088337,0
388572,1602651660000,1.3759,1.4000,1.1111,1.2493,987505.68,1602651719999,1.205528e+06,1392,614158.78,747451.518743,0
388573,1602651720000,1.2489,1.3593,1.1660,1.2552,549711.33,1602651779999,6.942747e+05,1103,277203.12,349648.438905,0
388574,1602651780000,1.2499,1.2500,1.1359,1.1742,343258.80,1602651839999,4.081314e+05,773,170461.39,202748.271257,0
388575,1602651840000,1.1754,1.2752,1.1753,1.2300,243382.88,1602651899999,2.965310e+05,440,170472.03,207510.362489,0
...,...,...,...,...,...,...,...,...,...,...,...,...
545226,1696118100000,1.1340,1.1340,1.1330,1.1330,431.60,1696118159999,4.891148e+02,6,112.00,127.008000,0
545227,1696118160000,1.1330,1.1340,1.1330,1.1330,32.20,1696118219999,3.649300e+01,4,10.40,11.793600,0
545228,1696118220000,1.1340,1.1340,1.1330,1.1330,38.50,1696118279999,4.364250e+01,3,22.00,24.948000,0
545229,1696118280000,1.1330,1.1340,1.1330,1.1340,28.10,1696118339999,3.185510e+01,2,17.80,20.185200,0


In [4]:
OHLC_COLUMNS = [
    "Open",
    "High",
    "Low",
    "Close"]
dataset = df.loc[:, OHLC_COLUMNS].astype(np.float64)
dataset = pd.DataFrame(dataset.to_numpy(), columns=OHLC_COLUMNS)


def add_features(df):
    df = df.copy()
    # add technical indicators to dataset
    df['SMA_256'] = df['Close'].rolling(window=256).mean()
    df['SMA_512'] = df['Close'].rolling(window=512).mean()
    df['SMA_1024'] = df['Close'].rolling(window=1024).mean()

    # convert SMA columns to distance in percentages from "Close"
    df['SMA_256'] = ((df['Close'] - df['SMA_256']) / df['SMA_256'])
    df['SMA_512'] = ((df['Close'] - df['SMA_512']) / df['SMA_512'])
    df['SMA_1024'] = ((df['Close'] - df['SMA_1024']) / df['SMA_1024'])

    # drop NaN rows resulting from the SMA calculations
    df.dropna(inplace=True)
    return df


dataset = add_features(dataset)
dataset

,Open,High,Low,Close,SMA_256,SMA_512,SMA_1024
1023,1.1284,1.1293,1.1284,1.1293,0.027575,-0.002864,-0.033380
1024,1.1284,1.1309,1.1284,1.1309,0.029032,-0.001275,-0.031812
1025,1.1310,1.1361,1.1310,1.1361,0.033741,0.003445,-0.027268
1026,1.1362,1.1364,1.1337,1.1364,0.033990,0.003865,-0.026914
1027,1.1375,1.1382,1.1356,1.1376,0.035056,0.005082,-0.025857
...,...,...,...,...,...,...,...
1556351,1.1340,1.1340,1.1330,1.1330,-0.003504,-0.005423,-0.004060
1556352,1.1330,1.1340,1.1330,1.1330,-0.003470,-0.005404,-0.004058
1556353,1.1340,1.1340,1.1330,1.1330,-0.003432,-0.005385,-0.004057
1556354,1.1330,1.1340,1.1330,1.1340,-0.002522,-0.004494,-0.003179


In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler


class MultiScaler:
    def __init__(self, min_max: MinMaxScaler, std: StandardScaler):
        self.min_max = min_max
        self.std = std


def preprocess(df: pd.DataFrame, scaler=None):
    # Apply percentage change only to OHLC columns
    df_pct = df[OHLC_COLUMNS].pct_change()

    # Clamp
    df_pct.clip(lower=-0.1, upper=0.1, inplace=True)

    # Concatenate the percentage-changed OHLC with the other columns
    df_all = pd.concat([df_pct, df.drop(columns=OHLC_COLUMNS)], axis=1)

    # Drop NA values (from pct_change operation)
    df_all.dropna(inplace=True)

    # Apply MinMax scaling to all columns
    if scaler is None:
        scaler = MultiScaler(MinMaxScaler(feature_range=(-1, 1), copy=False), StandardScaler(copy=False))
        df_multi_scaled = scaler.min_max.fit_transform(scaler.std.fit_transform(df_all))
    else:
        df_multi_scaled = scaler.min_max.transform(scaler.std.transform(df_all))

    df_scaled = pd.DataFrame(df_multi_scaled.copy(),
                             columns=df_all.columns,
                             index=df_all.index)

    return df_scaled, scaler


def invert_preprocess(original_start, scaler: MultiScaler, df):
    df = df.copy()

    original_start = original_start[OHLC_COLUMNS].to_numpy()
    # Invert MinMax scaling for all columns
    df_inv_scaled = pd.DataFrame(scaler.std.inverse_transform(scaler.min_max.inverse_transform(df.to_numpy())),
                                 columns=df.columns,
                                 index=df.index)

    # Recover the original OHLC values
    reversed_array = np.cumprod(1 + df_inv_scaled[OHLC_COLUMNS].to_numpy(), axis=0)
    reversed_array = reversed_array * original_start  # Scaling by the original_start to each element

    df_inv_scaled[OHLC_COLUMNS] = reversed_array

    return df_inv_scaled


preprocessed_dataset, scaler = preprocess(dataset)

# transform dataset so that all transform-invert transform pairs are idempotent
# dataset = invert_preprocess(dataset.iloc[0], scaler, preprocessed_dataset)
# dataset = add_features(dataset)

preprocessed_dataset

,Open,High,Low,Close,SMA_256,SMA_512,SMA_1024
1024,1.396588e-16,1.416807e-02,7.281434e-17,-0.003740,-0.246520,-0.283731,-0.298767
1025,2.304147e-02,4.598108e-02,2.304147e-02,0.028651,-0.238204,-0.275479,-0.291010
1026,4.597701e-02,2.640613e-03,2.387268e-02,-0.015477,-0.237763,-0.274744,-0.290407
1027,1.144165e-02,1.583949e-02,1.675928e-02,-0.007414,-0.235881,-0.272617,-0.288602
1028,7.032967e-03,5.798629e-02,7.044734e-03,0.046276,-0.224408,-0.261230,-0.277957
...,...,...,...,...,...,...,...
1556351,8.826125e-03,8.826125e-03,7.281434e-17,-0.018165,-0.303977,-0.290982,-0.251390
1556352,-8.818342e-03,1.082491e-16,7.281434e-17,-0.018165,-0.303917,-0.290949,-0.251387
1556353,8.826125e-03,1.082491e-16,7.281434e-17,-0.018165,-0.303850,-0.290916,-0.251385
1556354,-8.818342e-03,1.082491e-16,7.281434e-17,-0.009179,-0.302243,-0.289358,-0.249886


In [6]:
from sklearn.metrics import mean_absolute_error


def test_preprocess_invert_preprocess(original_df):
    preprocessed_df, scaler = preprocess(original_df)

    # Assume that 'original_start' is the first row of the original DataFrame
    original_start = original_df.iloc[0]

    inverted_df = invert_preprocess(original_start, scaler, preprocessed_df)

    mae_list = []
    for col in original_df.columns:
        # Start from the second row of the original_df for comparison
        mae = mean_absolute_error(original_df.iloc[1:][col], inverted_df[col])
        mae_list.append(mae)
        print(f"Mean Absolute Error for {col}: {mae}")

    avg_mae = sum(mae_list) / len(mae_list)
    print(f"Average MAE: {avg_mae}")

    return avg_mae < 1e-9


# Assuming 'dataset' is your original DataFrame
test_preprocess_invert_preprocess(dataset)


def test_orig_val():
    # The original_start passed to invert_preprocess() must be the first value in the corresponding
    # original DataFrame segment. For the first range, that's range_orig.iloc[0].
    # For the second range, it's range_orig.iloc[500].

    range_orig = dataset.iloc[2000:3000]
    range_preproc, s = preprocess(range_orig)

    # Inverted for the whole preprocessed range
    range1_inv = invert_preprocess(range_orig.iloc[0], s, range_preproc)

    # Inverted for the latter part of the preprocessed range
    range2_inv = invert_preprocess(range_orig.iloc[500], s, range_preproc.iloc[500:])

    # Due to floating point errors, equality may not be exact. So you might use pd.testing.assert_frame_equal
    # with the check_exact=False parameter
    pd.testing.assert_frame_equal(range1_inv.iloc[500:].reset_index(drop=True),
                                  range2_inv.reset_index(drop=True), check_exact=False)


test_orig_val()

Mean Absolute Error for Open: 1.126746228170941
Mean Absolute Error for High: 1.3774208346802361
Mean Absolute Error for Low: 0.8126206151996662
Mean Absolute Error for Close: 1.1374322405318922
Mean Absolute Error for SMA_256: 7.719346291693844e-18
Mean Absolute Error for SMA_512: 7.528852298967623e-18
Mean Absolute Error for SMA_1024: 6.083953981280191e-18
Average MAE: 0.636317131226105


In [7]:
import numpy as np
import pandas as pd


In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/model-weights/")
# except:
#   pass
# my_model.save(f"./drive/MyDrive/colab/model-weights/model-weights-{time.time()}.pt")

In [9]:
%load_ext tensorboard
%tensorboard --logdir tensorboard

Launching TensorBoard...

In [ ]:
from gymnasium import spaces
from stable_baselines3.common.preprocessing import get_flattened_obs_dim
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from torch import nn
import torch as th


class CustomExtractor(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: spaces.Dict, lstm_hidden_size=32, lstm_layers=1):
        super().__init__(observation_space, features_dim=1)

        extractors = {}
        total_concat_size = 0
        # We need to know size of the output of this extractor,
        # so go over all the spaces and compute output feature sizes
        for key, subspace in observation_space.spaces.items():
            if key == "prices_sequence":
                # Assuming sequence length is `seq_len` and feature number is `n_features`
                seq_len, n_features = subspace.shape
                extractors[key] = nn.LSTM(input_size=n_features, hidden_size=lstm_hidden_size, batch_first=True,
                                          num_layers=lstm_layers)
                total_concat_size += lstm_hidden_size  # hidden_dim is the LSTM output size
            elif key == "other":
                extractors[key] = nn.Flatten()
                total_concat_size += get_flattened_obs_dim(subspace)

        self.extractors = nn.ModuleDict(extractors)

        # Update the features dim manually
        self._features_dim = total_concat_size

    def forward(self, observations: th.Tensor) -> th.Tensor:
        encoded_tensor_list = []

        for key, extractor in self.extractors.items():
            obs_data = observations[key]
            if key == "prices_sequence":
                # LSTM expects input of shape (batch, seq_len, input_size)
                obs_data = obs_data.view(obs_data.size(0), -1, extractor.input_size)  # reshape if necessary
                out, _ = extractor(obs_data)
                # Using only the last output tensor (hidden state)
                encoded_tensor_list.append(out[:, -1, :])
            else:
                encoded_tensor_list.append(extractor(obs_data))

        return th.cat(encoded_tensor_list, dim=1)

In [10]:
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import random
import gymnasium as gym
from gymnasium import spaces
import torch

OBS_OTHER = "other"
OBS_PRICES_SEQUENCE = "prices_sequence"
MODEL_INPUT_IN_OBSERVATION = 256


def calculate_observation(df, scaler, buy_price):
    df_with_features = add_features(df)

    original_start = df_with_features.iloc[-1]
    df_preprocessed, _ = preprocess(df_with_features, scaler)

    curr_close = original_start.Close
    prev_close = df_with_features.iloc[-2].Close

    if buy_price is None:
        current_gain = 0
    else:
        current_gain = ((curr_close - buy_price) / buy_price)
    previous_prices = df_preprocessed.iloc[-MODEL_INPUT_IN_OBSERVATION:].to_numpy()  # shape = (N, 7)
    buy_status = 1 if buy_price is not None else 0
    observation = {
        OBS_PRICES_SEQUENCE: previous_prices.astype(np.float32),
        OBS_OTHER: np.concatenate([[buy_status], [current_gain]]).astype(np.float32)
    }

    return observation, curr_close, prev_close


class CustomEnv(gym.Env):
    SKIP_STEPS = 1024 + MODEL_INPUT_IN_OBSERVATION

    def __init__(self, dataset: pd.DataFrame, scaler: MultiScaler, episode_length=512,
                 commission=0.001):
        super().__init__()

        self.commission = commission

        # 0: hold; 1: buy; 2: sell
        self.action_space = spaces.Discrete(3)

        self.FEATURES_SHAPE = calculate_observation(dataset, scaler, None)[0]
        self.FEATURES_SHAPE = {k: v.shape for k, v in self.FEATURES_SHAPE.items()}
        print(f"obs length = {self.FEATURES_SHAPE}")

        # Update the observation space to include extra information
        self.observation_space = spaces.Dict(
            {OBS_PRICES_SEQUENCE: spaces.Box(low=-1, high=1,
                                             shape=self.FEATURES_SHAPE[OBS_PRICES_SEQUENCE],
                                             dtype=np.float32),
             OBS_OTHER: spaces.Box(low=-1, high=1,
                                   shape=self.FEATURES_SHAPE[OBS_OTHER],
                                   dtype=np.float32)})

        self.dataset = dataset
        self.scaler = scaler
        self.episode_length = episode_length
        self.current_step = 0
        self.start_index = 0
        self.buy_price = None

    def step(self, action):
        self.current_step += 1

        # Calculate the current observation
        observation, curr_close, prev_close = self.calculate_observation(self.current_step)

        # Initialize reward and info
        reward = 0

        # Give small rewards or penalties for holding
        if self.buy_price is not None:
            change = (curr_close - prev_close) / prev_close
            if abs(change) > 0.001:
                if change > 0:
                    reward += 0.01  # small reward for holding when price increases
                elif change < 0:
                    reward -= 0.015  # small penalty for holding when price decreases

        # Action logic
        if self.buy_price is None:
            if action == 1:  # Buy
                self.buy_price = curr_close
        else:
            sell_fee = curr_close * (1 - self.commission)
            buy_fee = self.buy_price * (1 + self.commission)
            gain_from_trade_fee = (sell_fee - buy_fee) / buy_fee
            if action == 2:  # Sell
                self.buy_price = None
                # Large reward for a profitable sell
                reward += gain_from_trade_fee * 4

        info = {}
        terminated = self.current_step >= self.episode_length
        return observation, reward, terminated, False, info

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.buy_price = None
        self.start_index = random.randint(self.SKIP_STEPS, len(self.dataset) - self.episode_length - 1)
        observation, _, _ = self.calculate_observation(self.current_step)
        info = {}
        return observation, info

    def render(self):
        pass

    def close(self):
        pass

    def calculate_observation(self, current_step):
        index_start = self.start_index + current_step - self.SKIP_STEPS
        index_end = self.start_index + current_step

        df = self.dataset.iloc[index_start:index_end]
        observation, curr_close, prev_close = calculate_observation(df, self.scaler, self.buy_price)

        return observation, curr_close, prev_close


from stable_baselines3.common.env_checker import check_env

env = CustomEnv(dataset, scaler)
check_env(env)
del env

env = make_vec_env(lambda: CustomEnv(dataset, scaler), n_envs=20, seed=42, vec_env_cls=SubprocVecEnv)

policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
                     features_extractor_class=CustomExtractor,
                     features_extractor_kwargs=dict(lstm_hidden_size=128, lstm_layers=2),
                     net_arch=[256, 256, 256])
# {'gamma': 0.8, 'ent_coef': 0.02, 'gae_lambda': 0.92}
rl_model = PPO("MultiInputPolicy", env,
               verbose=1,
               tensorboard_log="./tensorboard/",
               ent_coef=0.02,
               gae_lambda=0.92,
               gamma=0.8,
               policy_kwargs=policy_kvargs)
# rl_model.set_parameters("rl-model-best-model/best_model/best_model.zip")

obs length = {'prices_sequence': (256, 7), 'other': (2,)}


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation prices_sequence has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device


In [11]:
rl_model.policy

MultiInputActorCriticPolicy(
  (features_extractor): CustomExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(7, 128, num_layers=2, batch_first=True)
    )
  )
  (pi_features_extractor): CustomExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(7, 128, num_layers=2, batch_first=True)
    )
  )
  (vf_features_extractor): CustomExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(7, 128, num_layers=2, batch_first=True)
    )
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=130, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=256, bias=True)
      (5): LeakyReLU(negative

In [12]:
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="rl-model-checkpoints/",
    name_prefix="rl_model",
    verbose=1,
    save_vecnormalize=True,
    save_replay_buffer=True
)
eval_env = make_vec_env(lambda: CustomEnv(dataset, scaler, episode_length=2048), n_envs=1, seed=42,
                        vec_env_cls=SubprocVecEnv)
eval_callback = EvalCallback(eval_env, best_model_save_path="rl-model-best-model/best_model",
                             log_path="rl-model-best-model/results", eval_freq=50_000, verbose=1,
                             n_eval_episodes=5)

rl_model.learn(total_timesteps=1_000_000, callback=[checkpoint_callback, eval_callback])

Logging to ./tensorboard/PPO_44
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 512      |
|    ep_rew_mean     | -0.96    |
| time/              |          |
|    fps             | 1499     |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 40960    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 512         |
|    ep_rew_mean          | -0.881      |
| time/                   |             |
|    fps                  | 228         |
|    iterations           | 2           |
|    time_elapsed         | 359         |
|    total_timesteps      | 81920       |
| train/                  |             |
|    approx_kl            | 0.012497586 |
|    clip_fraction        | 0.18        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.396      

Process ForkServerProcess-59:
Process ForkServerProcess-60:
Process ForkServerProcess-61:
Process ForkServerProcess-52:
Process ForkServerProcess-55:
Process ForkServerProcess-56:
Process ForkServerProcess-47:
Process ForkServerProcess-48:
Process ForkServerProcess-49:
Process ForkServerProcess-43:
Process ForkServerProcess-57:
Process ForkServerProcess-50:
Process ForkServerProcess-44:
Process ForkServerProcess-58:
Process ForkServerProcess-53:
Process ForkServerProcess-51:
Process ForkServerProcess-54:
Process ForkServerProcess-41:
Process ForkServerProcess-45:
Process ForkServerProcess-46:
Process ForkServerProcess-42:

KeyboardInterrupt



In [ ]:
evaluations = np.load("rl-model-best-model/results/evaluations.npz")
evaluations_df = pd.DataFrame()
evaluations_df["timesteps"] = evaluations["timesteps"]
evaluations_df["results"] = evaluations["results"].mean(axis=1)
evaluations_df

In [ ]:
# backtesting

from backtesting import Backtest, Strategy, set_bokeh_output


class NeuralNetStrat(Strategy):
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.buy_price = None
        self.expected_gain = None

    def init(self):
        pass

    def next(self):
        if len(self.data) > 1200:
            df = self.data.df.iloc[-1200:].copy()
            df.drop(columns=["Volume"], inplace=True)
            observation, curr_close, _ = calculate_observation(df, scaler, self.buy_price)

            action, _ = rl_model.predict(observation, deterministic=True)
            if self.buy_price is None:
                if action == 1:
                    self.buy()
                    print(f"buy")
                    self.buy_price = curr_close
            else:
                if action == 2:
                    commission = 0.001
                    sell_fee = curr_close * (1 - commission)
                    buy_fee = self.buy_price * (1 + commission)
                    gain_from_trade_fee = (sell_fee - buy_fee) / buy_fee
                    print(f"gain {gain_from_trade_fee}")
                    self.sell()
                    self.buy_price = None


backtest_dataset = dataset[-20000:]
backtest_dataset["Volume"] = df[1023:]["Volume"]
bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,
              exclusive_orders=True)
stats = bt.run()
stats

In [ ]:
set_bokeh_output(notebook=False)
bt.plot()